In [19]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [20]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [21]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

In [22]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [23]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [42]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file = df_file.head(1)
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"


In [25]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [26]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,bakalım,8
4,değil,8
...,...,...
195,ister,1
196,hiç,1
197,hey,1
198,herkes,1


In [27]:
df_check = df_file_count[df_file_count["word_count"] > 3]
df_check

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,bakalım,8
4,değil,8
5,için,7
6,mi,7
7,çünkü,7
8,tamam,7
9,ne,6


In [28]:
check_list = df_check["word"].to_list()
check_list

['bir',
 'çok',
 'güzel',
 'bakalım',
 'değil',
 'için',
 'mi',
 'çünkü',
 'tamam',
 'ne',
 'kadar',
 'buraya',
 'böyle',
 'yani',
 'şey',
 'bu',
 'olsun',
 've',
 'oluyor',
 'var',
 'biraz',
 'şimdi',
 'daha',
 'gerçekten',
 'önce',
 'musun',
 'gibi',
 'geliyor',
 'gel',
 'ya',
 'zaman',
 'en',
 'ona',
 'hemen',
 'şekilde',
 'peki',
 'tam',
 'ben',
 'tekrar',
 'ama',
 'yok',
 'her',
 'aynı',
 'birlikte',
 'biri']

In [35]:
test = ["aet","önce"]

In [36]:
test in check_list

False

In [43]:
for i in range(len(df_file)):
    result_list = []
    var_sent = df_file.loc[i,"search_string"]
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for j in words:
        if j in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file.drop()
              
    
     

In [49]:
df_file.drop

0

In [44]:
result_list

[False, False, False, False]

In [45]:
all(result_list)

False

In [37]:
a = []

In [38]:
a.append(True)

In [40]:
a.append(False)

In [41]:
a

[True, False]